In [1]:
import fasttext
import numpy as np
import sys
sys.path.append('..')

In [2]:
import logging
log_format = '[%(asctime)s] [%(levelname)s] - %(message)s'
logging.basicConfig(level=logging.DEBUG, format=log_format)
log = logging.getLogger(__name__)

# writing to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
handler.setFormatter(log_format)
log.addHandler(handler)
logging.debug('hello')

[2020-01-27 22:35:24,432] [DEBUG] - hello


In [3]:
#model = fasttext.load_model('/home/legin/fasttext/wiki.en.bin')
model = fasttext.load_model('../test_lenin/lenin.bin')

In [6]:
m = model.f.getInputMatrix()

In [15]:
del m

In [4]:
import compressors.dumb  as d 
import storages.shelve_based as db
from importlib import reload
reload(d)

<module 'compressors.dumb' from '/home/legin/ft_compress/ft_compress/compressors/dumb.py'>

In [5]:

def hash_ft(string):
    h = np.array(2166136261, np.uint32)
    string = string.encode('utf-8')
    for string_i in string: 
        print (string_i)
        h = h^(string_i)
        h = h* 16777619
        print ('h', h)
    return h 
print (hash_ft('asd')%model.f.getArgs().bucket + len(model.words))

model.get_subwords('asd')#, hash_ft('<a>')%model.f.getArgs().bucket

97
h 36342608335481132
115
h 3757506531950928013
100
h 6856764573860679371
679973
/home/legin/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in long_scalars
  import sys


(['<as', '<asd', '<asd>', 'asd', 'asd>', 'sd>'],
 array([1429633, 1079763, 1504159,   63909,  485257,  753310]))

In [6]:
 len(model.words)

602

In [5]:
reload(d)
!rm test_db
dc = d.DumbCompressor(db.ShelveStorage('./test_db', 'c'))#~50% percents
dc.fit(model, 1)

[2020-01-27 22:35:33,420] [DEBUG] - loading ngrams
100%|██████████| 2000000/2000000 [00:43<00:00, 45912.76it/s]
[2020-01-27 22:36:16,991] [DEBUG] - loading words
100%|██████████| 602/602 [00:00<00:00, 28411.64it/s]
[2020-01-27 22:36:17,014] [DEBUG] - ready


In [30]:
words, idx  = model.get_subwords('Ленин')
for i, w in zip(idx, words):
    
    print (w, (dc.get_ngram_vector(w) - model.get_input_vector(i)).sum() )


Ленин 0.56496865
<Ле 0.0
<Лен 0.0
<Лени 0.0
<Ленин 0.0
Лен 0.0
Лени 0.0
Ленин 0.0
Ленин> 0.0
ени 0.0
енин 0.0
енин> 0.0
нин 0.0
нин> 0.0
ин> 0.0


In [32]:
#np.mean([model.get_input_vector(i) for i in idx], axis=0) - dc['ленин']#- dc['ленин']
model['Ленин'] - dc['Ленин']#- dc['ленин']

[2020-01-27 22:49:59,166] [DEBUG] - 0:3:<Ле
[2020-01-27 22:49:59,168] [DEBUG] - 1:3:Лен
[2020-01-27 22:49:59,168] [DEBUG] - 2:3:ени
[2020-01-27 22:49:59,169] [DEBUG] - 3:3:нин
[2020-01-27 22:49:59,169] [DEBUG] - 4:3:ин>
[2020-01-27 22:49:59,170] [DEBUG] - 0:4:<Лен
[2020-01-27 22:49:59,171] [DEBUG] - 1:4:Лени
[2020-01-27 22:49:59,172] [DEBUG] - 2:4:енин
[2020-01-27 22:49:59,172] [DEBUG] - 3:4:нин>
[2020-01-27 22:49:59,173] [DEBUG] - 0:5:<Лени
[2020-01-27 22:49:59,173] [DEBUG] - 1:5:Ленин
[2020-01-27 22:49:59,174] [DEBUG] - 2:5:енин>
[2020-01-27 22:49:59,174] [DEBUG] - 0:6:<Ленин
[2020-01-27 22:49:59,175] [DEBUG] - 1:6:Ленин>


array([ 0.0000000e+00, -2.9802322e-08, -1.4901161e-08,  7.4505806e-09,
       -7.4505806e-09,  0.0000000e+00, -1.4901161e-08, -4.6566129e-10,
        7.4505806e-09,  7.4505806e-09, -5.9604645e-08,  0.0000000e+00,
       -3.7252903e-09, -2.9802322e-08,  3.7252903e-09, -9.3132257e-10,
        7.4505806e-09,  1.4901161e-08,  0.0000000e+00,  0.0000000e+00,
       -9.3132257e-10,  2.9802322e-08, -1.8626451e-09, -3.7252903e-09,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
       -7.4505806e-09,  1.8626451e-09, -2.9802322e-08, -1.4901161e-08,
       -1.4901161e-08, -1.4901161e-08,  0.0000000e+00, -1.8626451e-09,
        0.0000000e+00,  0.0000000e+00, -1.4901161e-08, -4.6566129e-10,
       -7.4505806e-09,  1.4901161e-08, -7.4505806e-09,  0.0000000e+00,
       -1.4901161e-08, -7.4505806e-09,  3.7252903e-09, -1.4901161e-08,
        1.8626451e-09,  1.4901161e-08, -1.4901161e-08, -2.9802322e-08,
        0.0000000e+00, -2.9802322e-08, -2.9802322e-08,  0.0000000e+00,
      

In [70]:

hash_ft('<лен')%2000000+602#, d.hash_ft('<лен')+602

: 60
h 957132539
: -48
h 1544079025
: -69
h 674155198
: -48
h 743906346
: -75
h 3521008461
: -48
h 1006811943
: -67
h 565378670


1379272

In [17]:
word = 'ленин'
#dc[word] - model[word]
idx = model.get_subwords(word)
model.get_subwords(word)

(['<ле',
  '<лен',
  '<лени',
  '<ленин',
  'лен',
  'лени',
  'ленин',
  'ленин>',
  'ени',
  'енин',
  'енин>',
  'нин',
  'нин>',
  'ин>'],
 array([1009063, 1379272, 1123424,  106343,  597334, 1765262, 1608029,
        1183073, 1766703,  264864,  528194,  479407,  250699, 1992066]))

In [18]:
for k in dc.storage._db:
    print (k)


ngrams.нут
ngrams.есам>
ngrams.<Уль
word_ngrams.когда
ngrams.<пр
ngrams.того,>
ngrams.лне>
ngrams.<дан
ngrams.слов
ngrams.ных>
ngrams.<пе
word_ngrams.политики
word_ngrams.время
word_ngrams.—
ngrams.виками
ngrams.<дл
ngrams.ософия
ngrams.льш
ngrams.воен
ngrams.оль
ngrams.<Перв
ngrams.ую>
ngrams.мом
ngrams.<цель
ngrams.циклоп
ngrams.<Боль
word_ngrams.с.
ngrams.03>
ngrams.<обще
ngrams.<РСДРП
ngrams.ского>
ngrams.упска
ngrams.<ид
ngrams.сте
ngrams.изации
ngrams.<или>
ngrams.<свои
ngrams.азвит
ngrams.этог
word_ngrams.войны
ngrams.вля
ngrams.ности
ngrams.жна>
word_ngrams.пролетариат
ngrams.азной>
ngrams.<один
ngrams.сква:>
ngrams.артии>
ngrams.торог
ngrams.ссии,
ngrams.Гей
ngrams.написа
ngrams.тие
ngrams.1898>
ngrams.альног
ngrams.стичес
ngrams.без>
ngrams.<ли>
ngrams.<том
word_ngrams.которого
word_ngrams.РКП(б)
ngrams.ду>
ngrams.<наибо
ngrams.ный
ngrams.Центра
ngrams.1921>
ngrams.<О>
ngrams.<Плех
ngrams.чих
ngrams.л-де
ngrams.дек
ngrams.<доме>
ngrams.ак>
ngrams.ентра
ngrams.олити
ngrams.сс

In [29]:
model._words

['</s>',
 'в',
 'и',
 '—',
 'Ленин',
 'на',
 'В.',
 'с',
 'В',
 'И.',
 'не',
 'по',
 'Ленина',
 'что',
 'года',
 'о',
 'С.',
 'к',
 'А.',
 'из',
 'как',
 'от',
 'его',
 'он',
 'М.:',
 'Н.',
 '//',
 'для',
 'с.',
 'был',
 'ISBN',
 'за',
 'М.',
 'было',
 'а',
 '1917',
 'Владимир',
 'году',
 '№',
 '1918',
 '/',
 'или',
 'революции',
 'Т.',
 'Г.',
 'России',
 'были',
 'до',
 'По',
 'но',
 '·',
 'Ульянов',
 'это',
 'против',
 'Л.',
 'г.',
 'Ильич',
 'была',
 'время',
 'только',
 'же',
 'Ленин.',
 'их',
 'этом',
 'большевиков',
 'СССР',
 'П.',
 'Ю.',
 'во',
 'М.,',
 '1920',
 'под',
 'партии',
 'На',
 'апреля',
 'со',
 'Д.',
 'II',
 'после',
 'ни',
 'всех',
 'января',
 'своей',
 'Ф.',
 'так',
 'РСДРП',
 'С',
 'при',
 'Ленина,',
 'Ленин,',
 'её',
 'К.',
 'ЦК',
 'октября',
 'чтобы',
 'т.',
 'у',
 'все',
 'съезда',
 'истории',
 'вместе',
 'власти',
 'Совета',
 ':',
 'съезде',
 'После',
 'революция',
 'ему',
 'России.',
 'Советской',
 'об',
 'мы',
 'газеты',
 'года,',
 'одной',
 'бы',
 'где',
 'п

In [20]:
for i in zip(idx[0], idx[1]): 
    w = i[0]
    print (w)
    print ((dc.get_ngram_vector(w) - model.get_input_vector(i[1])).sum())

<ле
0.0
<лен


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [54]:
model.get_input_vector(1009063)
#dc.get_ngram_vector('<ле')

array([-0.06453893,  0.0930501 , -0.05321778, -0.04897046, -0.04192388,
       -0.06683032,  0.10498592, -0.00214278,  0.05364072,  0.0327807 ,
       -0.19478486,  0.01362339, -0.02251976, -0.03940681,  0.01539196,
       -0.01570239,  0.02207801,  0.03701511,  0.0394493 , -0.13102283,
       -0.02236869,  0.12979376, -0.01557101,  0.02558311, -0.01263936,
       -0.03850377,  0.06692874, -0.08996119, -0.04744707,  0.03577779,
       -0.12439587,  0.13861091, -0.05078376, -0.0286837 ,  0.02170644,
       -0.01497566,  0.0225643 , -0.07188074, -0.07700191, -0.00581786,
        0.06023261,  0.00487985,  0.08816288,  0.0175531 , -0.03513728,
       -0.0195913 ,  0.01878416, -0.05972658, -0.01932443,  0.11471016,
       -0.10450994, -0.12322816,  0.13805678,  0.17926702, -0.10862418,
        0.04115304, -0.05995381,  0.03707696, -0.09419206,  0.1186209 ,
       -0.0011181 ,  0.03387016,  0.1607741 , -0.10124911,  0.05376586,
       -0.00112932, -0.12756921,  0.09057014, -0.00435755, -0.03

In [10]:
len([dc])

1

In [9]:
import compressors.compressor8 as d 
from importlib import reload
reload(d)

<module 'compressors.compressor8' from '/home/legin/ft_compress/ft_compress/compressors/compressor8.py'>

In [10]:
dc = d.Compressor8bit(model, 2500000) #~50% percents

[2020-01-12 14:35:37,861] [DEBUG] - Loading words
[2020-01-12 14:35:48,702] [WARNING] - Too many ngrams, breaking
[2020-01-12 14:35:48,702] [WARNING] - Too many ngrams, breaking
[2020-01-12 14:35:48,702] [DEBUG] - Estimating parameters
[2020-01-12 14:36:15,707] [DEBUG] - filling matrix


KeyboardInterrupt: 

In [23]:
model.get_subwords('bananananana')[0]

['<ba',
 '<ban',
 '<bana',
 '<banan',
 'ban',
 'bana',
 'banan',
 'banana',
 'ana',
 'anan',
 'anana',
 'ananan',
 'nan',
 'nana',
 'nanan',
 'nanana',
 'ana',
 'anan',
 'anana',
 'ananan',
 'nan',
 'nana',
 'nanan',
 'nanana',
 'ana',
 'anan',
 'anana',
 'ananan',
 'nan',
 'nana',
 'nanan',
 'nanana',
 'ana',
 'anan',
 'anana',
 'anana>',
 'nan',
 'nana',
 'nana>',
 'ana',
 'ana>',
 'na>']

In [0]:
v = model.get_input_vector(0)

In [0]:
v2 = np.round(255*(v-dc.min_)/(dc.max_-dc.min_)).astype(np.uint8)


In [0]:
(dc.bytes_to_vec(dc.ngrams[',']) - v).sum()

In [0]:
dc.max_, v.max()

In [0]:
dc.ngrams

In [0]:
dc.info()